# Bring Your Own Container (BYOC)

In this notebook, we will cover how to bring our own container with either a framework or algorithm to train a model on SageMaker. 

We will use fastai in this case and build our container with custom training code integrated into the container. The other option is to use script mode which is easily done by changing the entrypoint.

Outline:
1. Build your own custom docker container with fastai framework
2. Register your custom docker container to Elastic Container Registry (ECR)
3. Use your custom docker container and provided training dataset to train your model


---
### 1. Build you customer Docker Container Image
Let's start with building a container image locally and then push that to ECR (Elastic Container Registry)

In [ ]:
%cd docker

In [ ]:
# let review the Dockerfile used to build your custom container iage
!pygmentize Dockerfile

In [ ]:
# let review training script
!pygmentize code/train.py

In [ ]:
# let review utils script
!pygmentize code/utils.py

In [ ]:
# build the docker image
!docker build -t fastai .

Add docker images architecture to explain what is going on
show the Dockerfile

In [ ]:
# show the docker image
!docker images

---
## 2. Register your custom docker image in ECR

Set the ecr details and tags - Lets set a few params here like ecr name space , tag name etc.

In [ ]:
from sagemaker import get_execution_role
import boto3
ecr_namespace = "sagemaker-training-containers/"
prefix = "byoc-fastai"

ecr_repository_name = ecr_namespace + prefix
role = get_execution_role()
account_id = role.split(":")[4]
region = boto3.Session().region_name
tag_name=account_id+'.dkr.ecr.'+region+'.amazonaws.com/'+ecr_repository_name+':latest'

In [ ]:
# review the tag name
tag_name

In [ ]:
# tag the image 
!docker tag fastai $tag_name

### ECR Repository and push steps

All of these can be scripted out but they are laid out this way for transparency and step evolution understanding

In [ ]:
# first login to ECR
!$(aws ecr get-login --no-include-email)

In [ ]:
# create a repository 
!aws ecr create-repository --repository-name $ecr_repository_name

In [ ]:
# now push the image to ECR
!docker push $tag_name

In [ ]:
# quick check 
container_image_uri = "{0}.dkr.ecr.{1}.amazonaws.com/{2}:latest".format( account_id, region, ecr_repository_name)
print(container_image_uri)

In [ ]:
# review the prepared training data
!aws s3 ls s3://2021-11-15-pvt-1234/train

---
### 3. Now Call your custom container to train the model

In [ ]:
# let train a model
import sagemaker
import json

role = sagemaker.get_execution_role()

# replace the s3_bucket name with your bucket name
s3_bucket = 's3://2021-11-15-pvt-1234/train'

# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}


hyperparameters = json_encode_hyperparameters({"lr":1e-03})

est = sagemaker.estimator.Estimator(
    container_image_uri,
    role,
    instance_count=1,
    #train_instance_type="local",  # we use local mode
    instance_type='ml.m5.4xlarge',
    base_job_name=prefix,
    hyperparameters=hyperparameters,
)

train_config = sagemaker.session.TrainingInput(s3_bucket)

est.fit({"train": train_config})

## Your model is now ready to be deployed